In [ ]:
!pip install -q pyarrow==15.0.1
!pip install -q datasets
!pip install -q langchain-groq
from datasets import Dataset, load_dataset
import pandas as pd
from tqdm.auto import tqdm
import time
from langchain_groq import ChatGroq
from collections import Counter

In [ ]:
dataset1=load_dataset("mediabiasgroup/mbib-base")

In [ ]:
# feature in dataset of huggingface( name of splits in dataset

print(dataset1)


In [ ]:
# A particular bias chosen at a time
dataset2=pd.DataFrame(dataset1['gender_bias'].shuffle(seed=42))


In [ ]:
# Chunk of 1000 datapoints
dataset3=dataset2[2000:3000]

In [ ]:
# import os
# os.environ['GROQ_API']='Your_API_Key'

In [ ]:
chat = ChatGroq(temperature=0,groq_api_key='Your_API_Key', model_name='llama-3.1-70b-versatile')

In [ ]:
import time
from tqdm import tqdm

pred = []
label = []
misclassified = []
max_retries = 5  # Set a limit for retries

for id, row in tqdm(dataset3.iterrows()):
    input = row['text']
    retries = 0  # Track the number of retries for each request
    prompt=f'Classify the following text as gender biased or unbiased. Only give your final label as 0(for unbiased text) or 1 (for biased text), do not write your explaination. Here is the text:{input}'

    while retries <= max_retries:
        try:
            response = chat.invoke(prompt)
            break  # Exit the retry loop if successful
        except Exception as e:
            retries += 1
            if retries > max_retries:
                print(f"Max retries reached for input: {input}. Skipping this entry.")
                response = None  # You could skip processing for this entry if retries fail
                break
            print(f"Error occurred: {e}. Retrying in {2 ** retries} seconds...")
            time.sleep(2 ** retries)  # Exponential backoff

    if response is None:
        continue  # Skip the rest if response is None

    print(input)
    print(response.content)
    print(row['label'])

    if response.content != str(row['label']):
        misclassified.append(row['text'])
        label.append(row['label'])

    pred.append(response.content)
    time.sleep(4)  # Control the rate of requests


In [ ]:
act=dataset3['label']

In [ ]:
len(act)

In [ ]:
# converting string labels to int type data
pred=[int(i) for i in pred]

In [ ]:
## result for corresponding chunk
from sklearn.metrics import classification_report
print(classification_report(act,pred))